In [7]:
import pandas as pd
import csv
import scipy
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2

In [8]:
def strip(s):
    return s.lstrip('"').rstrip('"')
def UTF16(s):
    ss=re.sub(r"\\u[0-9A-Fa-f]{4}",'',s)[:]
    return ss
def Leading(s):
    return s.lstrip()

In [9]:
train_raw = pd.read_csv('train-raw.tsv',sep = "\t", header = None,quoting=csv.QUOTE_NONE)
test_raw = pd.read_csv('dev-raw.tsv',sep = "\t", header = None,quoting=csv.QUOTE_NONE) #dev raw
answer_raw = pd.read_csv('test-raw.tsv',sep = "\t", header = None,quoting=csv.QUOTE_NONE)

train_raw.columns = ['id', 'Class','Twitter']
test_raw.columns = ['id', 'Class','Twitter']
answer_raw.columns = ['id', 'Class','Twitter']

train_raw['Twitter']=train_raw['Twitter'].map(strip).map(UTF16).map(Leading)
test_raw['Twitter']=test_raw['Twitter'].map(strip).map(UTF16).map(Leading)
answer_raw['Twitter']=answer_raw['Twitter'].map(strip).map(UTF16).map(Leading)

train_text = train_raw['Twitter']
test_text = test_raw['Twitter']
answer_text = answer_raw['Twitter']

train_label=train_raw['Class']
test_label=test_raw['Class']
answer_label=answer_raw['Class']


In [10]:
vectoriser = CountVectorizer(encoding='utf-8',decode_error='strict' )
X_train = vectoriser.fit_transform(train_text) #bags of words
X_test = vectoriser.transform(test_text ) 
X_answer=vectoriser.transform(answer_text ) 
'''k=100
x2 = SelectKBest(chi2, k=k)

X_train_x2 = x2.fit_transform(X_train,train_label)
X_test = x2.transform(X_test)
'''

'k=100\nx2 = SelectKBest(chi2, k=k)\n\nX_train_x2 = x2.fit_transform(X_train,train_label)\nX_test = x2.transform(X_test)\n'

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
dt=MultinomialNB(fit_prior= True, alpha= 1)   
dt.fit(X_train,train_label)
print(dt.score(X_test,test_label))


0.346446564476364


In [12]:
import pandas as pd
df = pd.DataFrame(dt.predict(X_answer))
df2=pd.DataFrame(answer_raw['id'])
df2['Class']=df
df2.to_csv('MultiNB.csv',index=False)
